# IBM Quantum Experience Lab - Lab script

Please use this notebook as a template for the work that you will submit for assessment for this lab exercise.

Student Names: Parth Bhandari
               Sheila Perez 
               Charlie Solomons-Tuke

## Warm up exercise

In [ ]:
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, Aer
import numpy as np

In [ ]:
qr = QuantumRegister(1)
cr = ClassicalRegister(1)

circuit = QuantumCircuit(qr, cr)

circuit.h(qr[0])
circuit.measure(qr[0],cr[0])

backend = Aer.get_backend('qasm_simulator')

exponent = np.arange(2,20,1) # 2^25 iterations is as much as I can calulate in a reasonable amound of time.
expectation = []
for s in [2**n for n in exponent]:
    job = execute(circuit, backend, shots=s)
    result=job.result().get_counts(circuit)
    expectation.append((result['0']-result['1'])/(result['0']+result['1']))

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot()
ax.scatter(exponent, expectation)
ax.plot(exponent, exponent*0, c=(0.7,.7,0.7))
ax.set_ylabel('<Z> expectation value')
ax.set_xlabel('log2(number of itterations)')

# Main exercise

In [2]:
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, Aer
from qiskit.tools.visualization import circuit_drawer
import numpy as np

In [3]:
qr = QuantumRegister(1)
cr = ClassicalRegister(1)
backend = Aer.get_backend('qasm_simulator')

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

## Pair 1:  𝜃=𝜋/4,  𝜙=0

When $\phi=0$ the state is simply $\ket{0}$.

For $\theta = \pi /4$, the observable in question is

$$ M(\pi/4) = \begin{bmatrix} 0 & \frac{1-i}{\sqrt{2}} \\ \frac{1+i}{\sqrt{2}} & 0
\end{bmatrix} = T X T^\dagger$$

### Method 2

We use the identity $M(\pi/4) = \frac{1}{\sqrt{2}} X + Y $, then estimate $X$ and $Y$ independently.

Below we define functions for measuring x and y given a circuit and a clasical bit to perform the measurement on.

In [4]:
def measure_z(circuit, c_bit, shots=1024):
    circuit.measure([0],c_bit)
    job = execute(circuit, backend, shots=shots)
    result = job.result().get_counts(circuit) 
    return (result['0']-result['1'])/(result['0']+result['1'])    

def measure_x(circuit, c_bit, shots=1024):
    circuit.h([0])
    circuit.measure([0],c_bit)
    job = execute(circuit, backend, shots=shots)
    result = job.result().get_counts(circuit) 
    return (result['0']-result['1'])/(result['0']+result['1'])

def measure_y(circuit, c_bit, shots=1024):
    circuit.s([0])
    circuit.s([0])
    circuit.s([0]) # using S^-1 = S^3
    circuit.h([0])
    circuit.measure([0],c_bit)
    job = execute(circuit, backend, shots=shots)
    result = job.result().get_counts(circuit) 
    return (result['0']-result['1'])/(result['0']+result['1'])

def measure_method_2(theta, qr, cr, shots=1024):
    xcircuit = QuantumCircuit(qr, cr)
    x = measure_x(xcircuit, cr[0], shots=shots)
    ycircuit = QuantumCircuit(qr, cr)
    y = measure_y(ycircuit, cr[0], shots=shots)
    M = np.cos(theta)*x + np.sin(theta)*y
    return M

In [13]:
M = measure_method_2(np.pi/4, qr, cr)

print(M)


0.015191747252054735


### Method 1

Note that $M = U(\theta)XU(\theta)^\dagger = U(\theta)HZHU(\theta)^\dagger$, so to measure M we just need to apply $HU(\theta)^\dagger$, and then measure $Z$. For $\theta = \pi/4$, $HU(\pi/4)^\dagger = HT^\dagger$

In [9]:
def measure_method_1(theta, qr, cr, shots=1024):
    zcircuit = QuantumCircuit(qr, cr)
    zcircuit.p(-theta, [0])
    zcircuit.h([0])
    M = measure_z(zcircuit, cr[0], shots=shots)
    return M

zcircuit = QuantumCircuit(qr, cr)
M = measure_method_1(np.pi/4, qr, cr)
print(M)

-0.03515625


## Pair 2:  𝜃=𝜋/8,  𝜙=𝜋/4

For $\phi = \pi/4$ the input state is $\frac{1}{\sqrt{2}}(\ket{0} + i\ket{1}) = \mathrm{exp}(-i\pi/4)HU(-\pi/2)H\ket{0}$

In [1]:
def init_circuit(phi, qr, cr):
    circuit = QuantumCircuit(qr, cr)
    circuit.h([0])
    circuit.u(0,0,-2*phi, [0])
    circuit.h([0]) # Ignoring global phase
    return circuit
    